# Demo Script for Milestone 10.15

## 0 Prepare task

### 0.1 prepare planning scene

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src')))

from pkg.global_config import RNB_PLANNING_DIR
from demo_utils.kiro_udp_send import start_mobile_udp_thread, send_pose_wait, get_xyzw_cur, get_reach_state
from pkg.utils.utils import *    
from pkg.utils.rotation_utils import *
from pkg.controller.combined_robot import *
from pkg.project_config import *
from demo_utils.streaming import *
from demo_utils.detect_table import *
from demo_utils.area_select import *
from pkg.detector.aruco.marker_config import get_aruco_map
aruco_map = get_aruco_map()


CONNECT_CAM = False # True
ENABLE_DETECT = False
DETECT_MARKER = True
CONNECT_INDY = False # True
CONNECT_MOBILE = False # True 
SHOW_MOTION_RVIZ = False

# Tool dimensions
TOOL_DIM = [0.32, 0.08]
TOOL_OFFSET = -0.04
ROBOT_Z_ANGLE = np.pi
MARGIN = 0
TRACK_THICKNESS = 0.001

INDY_BASE_OFFSET = (0.172,0,0.439)
ROBOT_Z_ANGLE = np.pi
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001

ip_cur =  get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"
CAM_HOST = '192.168.0.10'

print("Current PC IP: {}".format(ip_cur))
print("Mobile ROB IP: {}".format(MOBILE_IP))
print("CAM SERVER IP: {}".format(CAM_HOST))

from pkg.geometry.builder.scene_builder import SceneBuilder
from demo_utils.environment import *

sock_mobile, server_thread = start_mobile_udp_thread(recv_ip=ip_cur)

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP)
robot_config = RobotConfig(1, RobotType.indy7, (INDY_BASE_OFFSET, (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[False, CONNECT_INDY])

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.set_workspace_boundary(-3, 7, -5, 5, -CLEARANCE, 3, thickness=WALL_THICKNESS)


from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE

viewpoint = add_cam(gscene, tool_link=TIP_LINK)
add_indy_tool_kiro(gscene, tool_link=TIP_LINK, face_name=TOOL_NAME, zoff=TOOL_OFFSET)

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

# Set planner
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False, incremental_constraint_motion=True)
mplan.motion_filters = [GraspChecker(pscene)]
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

# Register binders
from pkg.planning.constraint.constraint_actor import VacuumTool, Gripper2Tool, PlacePlane, SweepFramer, WayFramer
brush_face = pscene.create_binder(bname=TOOL_NAME, gname=TOOL_NAME, _type=SweepFramer, 
                                  point=(-gscene.NAME_DICT['brush_face'].dims[0]/2-CLEARANCE,0,0), 
                                  rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,WALL_THICKNESS)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2-CLEARANCE), rpy=(0,0,0))

Current PC IP: 192.168.0.123
Mobile ROB IP: 192.168.0.102
CAM SERVER IP: 192.168.0.10
connection command:
kmb0: False
indy1: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


### 0.2 Wait task start queue

## 1. Bed cleaning

### 1.1 Detect bed

#### 1.1.1 Move to bed-seek pose 

In [2]:
VIEW_POSE = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
VIEW_LOC = [0,]*6
VIEW_POSE_EXT = np.array(VIEW_LOC + list(VIEW_POSE))
if CONNECT_INDY:
    with indy:
        indy.joint_move_to(np.rad2deg(VIEW_POSE))
        time.sleep(0.5)
        indy.wait_for_move_finish()
        Qcur = np.deg2rad(indy.get_joint_pos())
else:
    Qcur = VIEW_POSE
gscene.show_pose(VIEW_POSE_EXT)

#### 1.1.2  detect bed and add to the scene

In [3]:
bed_center = (2,0,0)
bed_rpy = (0,0,np.pi/2)
COLOR_BED_COL = (0,1,0,0.3)

bed_mat = add_bed(gscene, bed_center, bed_rpy, COLOR_BED_COL)

closet_left, closet_rightup, closet_rightdown = add_closet(
    gscene, closet_center=np.matmul(Rot_rpy(bed_rpy), (-0.75,-1,0))+bed_center, closet_rpy=bed_rpy, 
    COLOR_CLOSET_COL=(0,1,0,0.3))

### 1.2 Make bed cleaning plan

#### 1.2.1 Select cleaning locations

In [4]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_actor import *
from pkg.planning.constraint.constraint_subject import *

In [5]:
wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, (0,0,0), (0,0,0), 
                                    parent="floor_ws", color=(0, 0, 1, 0.5))
gcheck = GraspChecker(pscene)
ccheck = CachedCollisionCheck(gcheck, wp_task, wp_hdl, wayframer)

In [6]:
BED_OFFSET = 0.05
TABLE_DIMS = bed_mat.dims[1], bed_mat.dims[0], bed_mat.dims[2]
T_e_brush = brush_face.get_tf_handle(crob.home_dict, from_link=TIP_LINK)
T_brush_e = SE3_inv(T_e_brush)
EE_DEPTH_OFF = T_brush_e[0, 3]
EE_HEIGHT = round(bed_mat.get_tf(HOME_DICT)[2,3] + bed_mat.dims[2]/2, 5) + T_brush_e[2, 3] + BED_OFFSET - INDY_BASE_OFFSET[2]

In [7]:
div_base_dict, Tsm_keys, surface_div_centers, sqdiv_size, div_num, (ax_step, ax_swp, ax_pln) = \
                        get_division_dict(bed_mat, brush_face, robot_config, 
                                          plane_val=EE_HEIGHT, tip_dir=None, TOOL_DIM=TOOL_DIM, 
                                          ccheck=ccheck)
idx_bases, idc_divs, covered_all = select_max_cover_bases(
    div_base_dict, Tsm_keys, surface_div_centers, div_num, ax_step)

res: False (0.0)
kmb0_platform_Box_1 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_1 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_1 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_1 - closet_left
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_left
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_left
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_left
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_left
res: False (0.0)
kmb0_platform_Box_1 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_left
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - closet_rightdown
res: False (0.0)
kmb0_p

res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_1 - bed_head
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_1 - bed_head
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_1 - bed_head
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_1 - bed_head
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Box_2 - bed_head
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylin

res: False (0.0)
kmb0_platform_Box_2 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_2 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_2 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Box_3 - closet_left


res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_3 - bed_head
res: True 

res: False (0.0)
kmb0_platform_Box_3 - bed_head
res: False (0.0)
kmb0_platform_Box_3 - bed_head
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_2 - bed_head
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_2 - bed_head
res: False (0.0)
kmb0_platform_Box_3 - bed_head
res: False (0.0)
kmb0_platform_Box_3 - bed_head
res: False (0.0)
kmb0_platform_Box_2 - bed_head
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)

res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: True (0.0025)
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Cylinder_0 - bed_mat
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Box_2 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_left
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - closet_rightdown
res: False (0.0)
kmb0_platform_Box_3 - bed_mat
res: False (0.0)
kmb0_platform_Box_

## add waypoints

In [8]:
def set_base_sweep(floor_gtem, Tsm, surface, swp_centers, WP_DIMS,  Q_dict):
    Tbf = floor_gtem.get_tf(Q_dict)
    Tbs = surface.get_tf(Q_dict)
    Tbm = np.matmul(Tbs, Tsm)
    Tfm = np.matmul(SE3_inv(Tbf), Tbm)
    Tfm[2,3] = 0
    wp_task, wp_hdl = add_waypoint_task(pscene, "waypoint", WP_DIMS, Tfm[:3,3], Rot2rpy(Tfm[:3,:3]), 
                                        parent=floor_gtem.name)

    ax_swp_tool = 1
    TOOL_DIM_SWEEP = TOOL_DIM[ax_swp_tool]/2
    swp_min, swp_max, ax_swp = get_min_max_sweep_points(
        swp_centers, sqdiv_size, TOOL_DIM_SWEEP)
    sweep_task = add_sweep_task(pscene, "sweep", surface, swp_min, swp_max, Tsm, wp_dims=TOOL_DIM)

In [9]:
crob.home_pose = np.array([0,] * 12)
crob.home_dict = list2dict(crob.home_pose, gscene.joint_names)

In [10]:
# remove finished highlights
for i_b in idx_bases:
    hl_name = "sweep_fin_{}".format(i_b)
    if hl_name in gscene.NAME_DICT:
        gscene.remove(gscene.NAME_DICT[hl_name])

floor_ws = gscene.NAME_DICT["floor_ws"]    
mplan.motion_filters = [gcheck]
Qcur = crob.home_pose
for i_b, idc_div in zip(idx_bases, idc_divs):
    swp_centers = np.array(surface_div_centers)[idc_div]
    Tsm = T_xyzquat(Tsm_keys[i_b])
    set_base_sweep(floor_ws, Tsm, bed_mat, swp_centers, WP_DIMS,  crob.home_dict)

    ppline.set_motion_planner(mplan)
    ppline.set_task_planner(tplan)
    initial_state = pscene.initialize_state(Qcur)

    ppline.search(initial_state, [(2, 1)], verbose=True,
                  timeout=0.3, timeout_loop=10, multiprocess=True,
                  add_homing=False, terminate_on_first=True, post_optimize=False)
    snode_schedule = tplan.get_best_schedule(at_home=False)
    snode_last = snode_schedule[-1]
    ref_state = snode_last.state.copy(pscene)
    ref_state.Q[crob.idx_dict[ROBOT_NAME]] = crob.home_pose[crob.idx_dict[ROBOT_NAME]]
    snode_schedule = snode_schedule+ppline.add_return_motion(snode_last, ref_state)
    ppline.play_schedule(snode_schedule)
    Qcur = snode_schedule[-1].state.Q
    Qcur[3:6] = 0
    # leave highlight on cleared area
    swp_fin = gscene.copy_from(gscene.NAME_DICT["sweep"], new_name="sweep_fin_{}".format(i_b), color=(0,0,1,0.5))
    swp_fin.dims = (swp_fin.dims[0], swp_fin.dims[1], swp_fin.dims[2]+0.002)
    gscene.update_marker(swp_fin)

Use 36/36 agents
try: 0 - (0, 0)->(0, 1)
res: True (0.002)
try: 0 - (0, 0)->(1, 0)
try: 0 - (0, 0)->(0, 1)
try transition motion
res: True (0.011)
res: True (0.002)
try: 0 - (0, 0)->(0, 1)
try transition motion
try: 0 - (0, 0)->(1, 0)
res: True (0.002)
try transition motion
try transition motion
try: 0 - (0, 0)->(0, 1)
res: True (0.011)
try: 0 - (0, 0)->(1, 0)
res: True (0.002)
try transition motion
try: 0 - (0, 0)->(1, 0)
res: True (0.011)
transition motion tried: True
try transition motion
try: 0 - (0, 0)->(0, 1)
try transition motion
result: 0 - (0, 0)->(0, 1) = success
res: True (0.011)
try: 1 - (0, 1)->(1, 1)
branching: 0->1 (0.13/10.0 s, steps/err: 11(75.7558345795 ms)/0.00126668479634)
res: True (0.002)
try transition motion
try: 1 - (0, 1)->(1, 1)
transition motion tried: True
res: True (0.011)
try: 0 - (0, 0)->(0, 1)
res: True (0.002)
result: 0 - (0, 0)->(0, 1) = success
try transition motion
try transition motion
res: True (0.011)
transition motion tried: True
try: 2 - (0, 1)

branching: 0->21 (0.37/10.0 s, steps/err: 11(94.4550037384 ms)/0.00194856534094)
try transition motion
try: 21 - (0, 1)->(1, 1)
try constrained motion
try transition motion
try: 20 - (1, 1)->(2, 1)
try: 17 - (0, 1)->(1, 1)
try transition motion
try: 18 - (1, 1)->(2, 1)
try: 19 - (0, 1)->(1, 1)
try: 6 - (0, 1)->(1, 1)
res: True (0.0092)
try: 0 - (0, 0)->(1, 0)
res: True (0.011)
res: True (0.011)
res: True (0.011)
res: True (0.0092)
res: True (0.011)
transition motion tried: True
try: 4 - (0, 1)->(1, 1)
transition motion tried: True
result: 1 - (0, 1)->(1, 1) = success
res: True (0.011)
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
try transition motion
transition motion tried: True
try transition motion
try constrained motion
branching: 1->22 (0.41/10.0 s, steps/err: 53(136.104106903 ms)/0.00116259269184)
try constrained motion
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
result: 6 - (0, 1)->(1, 1) = success
result: 9 - (0, 1)->(1, 1) = succes

Motion Plan Failure
try: 38 - (0, 1)->(1, 1)
result: 0 - (0, 0)->(1, 0) = fail
try: 23 - (0, 1)->(1, 1)
branching: 23->36 (0.57/10.0 s, steps/err: 53(112.154960632 ms)/0.00162436388075)
result: 0 - (0, 0)->(0, 1) = success
res: True (0.011)
try: 39 - (0, 1)->(1, 1)
branching: 0->39 (0.59/10.0 s, steps/err: 11(83.6029052734 ms)/0.00188511888462)
res: True (0.011)
try transition motion
try: 0 - (0, 0)->(1, 0)
transition motion tried: True
res: True (0.011)
result: 14 - (0, 1)->(1, 1) = success
res: True (0.011)
try transition motion
branching: 14->40 (0.62/10.0 s, steps/err: 48(151.251077652 ms)/0.000857184386694)
try transition motion
transition motion tried: True
try: 40 - (1, 1)->(2, 1)
try: 0 - (0, 0)->(0, 1)
res: True (0.002)
result: 1 - (0, 1)->(1, 1) = success
res: True (0.0092)
try transition motion
try transition motion
branching: 1->41 (0.64/10.0 s, steps/err: 48(121.237039566 ms)/0.0013165759797)
try constrained motion
try: 41 - (1, 1)->(2, 1)
transition motion tried: True
res

result: 0 - (0, 0)->(0, 1) = success
try constrained motion
branching: 0->54 (1.2/10.0 s, steps/err: 11(103.77907753 ms)/0.00159111379046)
result: 18 - (1, 1)->(2, 1) = fail
try transition motion
try transition motion
try: 54 - (0, 1)->(1, 1)
try: 21 - (0, 1)->(1, 1)
transition motion tried: True
res: True (0.011)
joint min
res: True (0.011)
constrained motion tried: False
joint max
try transition motion
constrained motion tried: False
Motion Plan Failure
try transition motion
result: 47 - (0, 1)->(1, 1) = success
Motion Plan Failure
result: 20 - (1, 1)->(2, 1) = fail
branching: 47->55 (1.24/10.0 s, steps/err: 53(149.44601059 ms)/0.00111707109904)
try: 55 - (1, 1)->(2, 1)
result: 27 - (1, 1)->(2, 1) = fail
res: True (0.0092)
try: 38 - (0, 1)->(1, 1)
try: 15 - (0, 1)->(1, 1)
transition motion tried: True
try constrained motion
res: True (0.011)
result: 0 - (0, 0)->(0, 1) = success
joint max
joint max
try transition motion
constrained motion tried: False
transition motion tried: True
res

branching: 60->69 (1.52/10.0 s, steps/err: 41(114.855051041 ms)/0.00147665025696)
try: 68 - (1, 1)->(2, 1)
result: 0 - (0, 0)->(0, 1) = success
branching: 0->70 (1.54/10.0 s, steps/err: 11(70.1549053192 ms)/0.00147967890209)
res: True (0.011)
try: 70 - (0, 1)->(1, 1)
res: True (0.011)
try: 69 - (1, 1)->(2, 1)
res: True (0.0092)
try transition motion
transition motion tried: True
res: True (0.011)
try transition motion
try transition motion
transition motion tried: True
try constrained motion
res: True (0.0092)
joint min
try: 57 - (0, 1)->(1, 1)
transition motion tried: True
result: 47 - (0, 1)->(1, 1) = success
result: 0 - (0, 0)->(0, 1) = success
result: 56 - (0, 1)->(1, 1) = success
res: True (0.011)
constrained motion tried: False
branching: 0->73 (1.57/10.0 s, steps/err: 11(107.604026794 ms)/0.00144102608356)
branching: 56->72 (1.57/10.0 s, steps/err: 53(132.300138474 ms)/0.000271237900056)
Motion Plan Failure
result: 45 - (1, 1)->(2, 1) = fail
branching: 47->71 (1.57/10.0 s, steps

...

res: True (0.0092)
try transition motion
transition motion tried: True
joint max
try: 0 - (0, 0)->(1, 0)
try: 369 - (0, 1)->(1, 1)
result: 255 - (0, 1)->(1, 1) = success
branching: 255->370 (9.41/10.0 s, steps/err: 67(575.073957443 ms)/0.00135159313368)
constrained motion tried: False
Motion Plan Failure
res: True (0.0092)
result: 356 - (1, 1)->(2, 1) = fail
try: 370 - (1, 1)->(2, 1)
res: True (0.0092)
transition motion tried: True
try transition motion
try: 0 - (0, 0)->(1, 0)
try transition motion
result: 0 - (0, 0)->(0, 1) = success
res: True (0.0092)
res: True (0.011)
transition motion tried: True
branching: 0->371 (9.44/10.0 s, steps/err: 87(140.57302475 ms)/0.00184233084785)
try transition motion
try: 371 - (0, 1)->(1, 1)
result: 360 - (0, 1)->(1, 1) = success
transition motion tried: False
try constrained motion
Motion Plan Failure
branching: 360->372 (9.46/10.0 s, steps/err: 56(386.732816696 ms)/0.00207483144642)
result: 0 - (0, 0)->(1, 0) = fail
res: True (0.0092)
try: 372 - (1

transition motion tried: True
try transition motion
try: 384 - (1, 1)->(2, 1)
result: 376 - (0, 1)->(1, 1) = success
try: 385 - (0, 1)->(1, 1)
res: True (0.011)
res: True (0.011)
transition motion tried: True
res: True (0.0092)
res: True (0.011)
try constrained motion
try constrained motion
branching: 376->386 (9.73/10.0 s, steps/err: 48(156.461000443 ms)/0.000943446785465)
result: 51 - (0, 1)->(1, 1) = success
transition motion tried: True
try constrained motion
try transition motion
branching: 51->387 (9.75/10.0 s, steps/err: 55(223.521947861 ms)/0.0017033980904)
try: 387 - (1, 1)->(2, 1)
try: 386 - (1, 1)->(2, 1)
res: True (0.011)
result: 237 - (0, 1)->(1, 1) = success
transition motion tried: False
res: True (0.011)
branching: 237->388 (9.78/10.0 s, steps/err: 51(210.631847382 ms)/0.00110848955023)
try constrained motion
Motion Plan Failure
joint max
constrained motion tried: False
try: 388 - (1, 1)->(2, 1)
transition motion tried: True
result: 0 - (0, 0)->(1, 0) = fail
Motion Plan

======================= terminated 0: max iteration time reached (1633428623/1633428613.45 s) ===============================
======================= terminated 26: max iteration time reached (1633428623/1633428613.45 s) ===============================
transition motion tried: True
result: 251 - (0, 1)->(1, 1) = success
branching: 251->398 (10.09/10.0 s, steps/err: 59(272.588968277 ms)/0.00120370561704)
======================= terminated 21: max iteration time reached (1633428623/1633428613.45 s) ===============================
transition motion tried: True
transition motion tried: True
transition motion tried: True
result: 0 - (0, 0)->(0, 1) = success
branching: 0->400 (10.11/10.0 s, steps/err: 80(175.476074219 ms)/0.00205211249577)
result: 0 - (0, 0)->(0, 1) = success
result: 270 - (0, 1)->(1, 1) = success
branching: 270->401 (10.11/10.0 s, steps/err: 45(96.5540409088 ms)/0.00201709257247)
======================= terminated 33: max iteration time reached (1633428623/1633428613.45 s) 

result: 375 - (1, 1)->(2, 1) = fail
======================= terminated 35: max iteration time reached (1633428623/1633428613.45 s) ===============================
======================= terminated 10: max iteration time reached (1633428623/1633428613.45 s) ===============================
transition motion tried: False
Motion Plan Failure
result: 0 - (0, 0)->(1, 0) = fail
======================= terminated 3: max iteration time reached (1633428623/1633428613.45 s) ===============================
collision
constrained motion tried: False
Motion Plan Failure
result: 378 - (1, 1)->(2, 1) = fail
======================= terminated 13: max iteration time reached (1633428623/1633428613.45 s) ===============================
joint max
constrained motion tried: False
Motion Plan Failure
joint max
result: 384 - (1, 1)->(2, 1) = fail
constrained motion tried: False
Motion Plan Failure
result: 386 - (1, 1)->(2, 1) = fail
======================= terminated 8: max iteration time reached (1633428623/1

IndexError: list index out of range

In [32]:
gcheck.chain_dict

{'indy1': {'joint_names': ['indy1_joint0',
   'indy1_joint1',
   'indy1_joint2',
   'indy1_joint3',
   'indy1_joint4',
   'indy1_joint5'],
  'link_names': ['indy1_link0',
   'indy1_link1',
   'indy1_link2',
   'indy1_link3',
   'indy1_link4',
   'indy1_link5',
   'indy1_link6',
   'indy1_tcp'],
  'tip_link': 'indy1_tcp'},
 'kmb0': {'joint_names': ['kmb0_joint0',
   'kmb0_joint1',
   'kmb0_joint2',
   'kmb0_joint3',
   'kmb0_joint4',
   'kmb0_joint5'],
  'link_names': ['kmb0_link0',
   'kmb0_link1',
   'kmb0_link2',
   'kmb0_link3',
   'kmb0_link4',
   'kmb0_link5',
   'kmb0_platform'],
  'tip_link': 'kmb0_platform'}}

In [15]:
lname = 'kmb0_platform'
children = map(lambda x: x[1], gscene.urdf_content.child_map[lname])


{'indy1': 'indy1_link0', 'kmb0': 'kmb0_link0'}

In [19]:

children

['indy1_link0']

In [11]:
gcheck.robot_ex_link_dict

{'indy1': ['base_link',
  'kmb0_link0',
  'kmb0_link1',
  'kmb0_link2',
  'kmb0_link3',
  'kmb0_link4',
  'kmb0_link5',
  'kmb0_platform'],
 'kmb0': ['base_link',
  'indy1_link0',
  'indy1_link1',
  'indy1_link2',
  'indy1_link3',
  'indy1_link4',
  'indy1_link5',
  'indy1_link6',
  'indy1_tcp']}

## TODO
* 실제 동작 생성 조건 추가하기

In [ ]:

sweep_width, (area_width, area_depth), width_range, divisions, div_num, area_center = select_task_area(
    robot_config, TABLE_DIMS, TOOL_DIM, EE_DEPTH_OFF, EE_HEIGHT, ROBOT_Z_ANGLE, 
    MARGIN=MARGIN, TOOL_DEPTH_MIN=0.6, TOOL_DEPTH_MAX=1.0, allow_back_approach=True)

In [ ]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

table_name = "bed_mat"

# floor
floor = pscene.create_binder(bname="floor", gname="floor_ws", _type=PlacePlane, point=None)

# waypoints
TABLE_DIM_DW = np.array(TABLE_DIMS[:2])
track_size_dw = TABLE_DIM_DW / (divisions[1], divisions[0])
REF_CENTER_00 = (TABLE_DIM_DW/2 - track_size_dw / 2) * (-1,1)

Trt = SE3(Rot_axis(3,np.pi), tuple(np.negative(area_center))+(TABLE_DIMS[2]-TRACK_THICKNESS/2,)) # offset from indy base to selected track
Tmr = get_tf(to_link=ROBOT_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content, from_link=MOBILE_BASE) # from mobile to indy
Ttm = SE3_inv(np.matmul(Tmr, Trt)) # from track to mobile
Tfb = SE3_inv(gscene.NAME_DICT["floor_ws"].get_tf(HOME_DICT)) # from floor to base_link
Tfm_home = np.matmul(Tfb, get_tf(to_link=MOBILE_BASE, joint_dict=HOME_DICT, urdf_content=gscene.urdf_content)) # from floor to mobile home

T_tab_trac_list = []
track_list = []
wp_list = []
for i_d in range(divisions[1]):
    for i_w in range(divisions[0]):
        xyz = (tuple((REF_CENTER_00 + track_size_dw * (i_d, -i_w)) * ((1, -1) if i_d == 1 else (1, 1)))
               +(TABLE_DIMS[2]/2-TRACK_THICKNESS/2,))
        rpy = (0,0,np.pi) if i_d == 1 else (0,0,0)
        track_tmp = gscene.create_safe(GEOTYPE.BOX, "track_{}_{}".format(i_d, i_w), "base_link",
                                       dims = tuple(track_size_dw)+(TRACK_THICKNESS,), center = xyz, rpy = rpy, 
                                       color=(0.0,0.0,0.8,0.3), display=True, fixed=True, collision=False, 
                                       parent=table_name)
        track_list.append(track_tmp)
        Tft = np.matmul(Tfb, track_tmp.get_tf(HOME_DICT))
        Tfm = np.matmul(Tft, Ttm)
        xyz, rpy = T2xyzrpy(Tfm)
        wp_list.append(
            gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(len(wp_list)), link_name="base_link",
                               dims=WP_DIMS, center=tuple(xyz[:2])+(0,), 
                               rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, 
                               parent="floor_ws")
        )

#### 1.2.2 Generate motions to cleaning locations

#### 1.2.3 Generate cleaning sweep motions

### 1.3 Execute bed cleaning sequence


## 2. Closet cleaning

### 2.1. Detect closet

#### 2.1.1 Move to closet side of bed

#### 2.1.2 Detect the closet and add to scene

In [ ]:
closet_center=(3,-0.75,0), closet_rpy=(0,0,np.pi/2)

add_closet(COLOR_CLOSET_COL=(0,1,0,0.3))

### 2.2. Make closet cleaning plan

#### 2.2.1 Select cleaning location

#### 2.2.2 Generate cleaning sweep motions

## 3. Return to home